In [24]:
# Multiple Participants -- Approach 2: Model Template
# Train & Test on Distinct Groups of Participants
# Approach 2: Use Small % of Test Participant's Data (Chronological Split)

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations
import itertools
import json
pd.options.mode.chained_assignment = None

In [26]:
# Key Features: op_num_people, of_pose_distance, of_gaze_distance, of_confidence, ros_mistakes_session, ros_ts_robot_talked, timestamp
use_key_features = False

# Leave One Out Run: test on each participant & train on remaining
leave_one_out_run = False

# Number of Estimators for Bagging
num_estimators = 10

# Sample Percentage for Bagging
sample_percentage = 0.75

# Percentage of Test Participant Data to Use
test_percentage_to_use = 0.3

# File Name for Model Results
output_file = 'multi3_af_nn_results_val.csv'

# 1 -- Load Data

In [27]:
# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p5_master_window.csv'
p7_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p7_master_window.csv'
p9_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p9_master_window.csv'
p11_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p11_master_window.csv'
p12_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p12_master_window.csv'
p17_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p17_master_window.csv'
p18_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p18_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data9 = pd.read_csv(p9_file)
data11 = pd.read_csv(p11_file)
data12 = pd.read_csv(p12_file)
data17 = pd.read_csv(p17_file)
data18 = pd.read_csv(p18_file)

In [28]:
data = [data5, data7, data9, data11, data12, data17, data18]
part = [5, 7, 9, 11, 12, 17, 18]

In [29]:
remove = [
'engagement_change',
'ros_GAME_STATE',
'ros_PARTICIPANT_STATE',
'ros_ROBOT_STATE',
'ros_activity',
'ros_diff_1_change',
'ros_diff_2_change',
'ros_diff_3_change',
'ros_diff_4_change',
'ros_diff_5_change',
'ros_difficulty',
'ros_skill_EM_change',
'ros_skill_NC_change',
'ros_skill_OS_change',
'ros_ts_attempt_var',
'ros_games_session_change',
'ros_in_game_change',
'ros_mistakes_session_change',
'ros_mistakes_game_change',
'ros_skill',
'ros_ts_game_start_var',
'ros_ts_robot_talked_var',
'ros_game_correct',
'ros_game_incorrect',
'ros_game_start',
'ros_mistake_made',
]

for i, d in enumerate(data):
    data[i] = d.drop(columns=remove)

In [30]:
# Remove rows where engagment NaN
for i,d in enumerate(data):
    data[i] = d[np.isfinite(d['engagement'])]
    
# Remove rows where engagment is -1
for i,d in enumerate(data):
    data[i] = d[d['engagement']>=0]

# 2 -- Choose Feature Set

Feature Dictionary: https://docs.google.com/spreadsheets/d/1ewoVPHwW68Ins0AOVZf-0lsl_wW0_ZzuByuDiNJETBY/edit?usp=sharing

### Data Overview

In [31]:
# Main Columns
basic_cols = []
for i in data[0].columns:
    if 'of_' not in i and 'op_' not in i and 'ros_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
for i in basic_cols:
    print(i)

engagement
participant
session_num
timestamp


In [32]:
# Open Face Columns

of_cols = []
for i in data[0].columns:
    if ('of_' in i or 'op_' in i) and '_change' not in i and '_var' not in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
for i in of_cols:
    print(i)

of_AU01_c
of_AU02_c
of_AU04_c
of_AU05_c
of_AU06_c
of_AU07_c
of_AU09_c
of_AU10_c
of_AU12_c
of_AU14_c
of_AU15_c
of_AU17_c
of_AU20_c
of_AU23_c
of_AU25_c
of_AU26_c
of_AU28_c
of_AU45_c
of_confidence
of_gaze_0_x
of_gaze_0_y
of_gaze_0_z
of_gaze_1_x
of_gaze_1_y
of_gaze_1_z
of_gaze_angle_x
of_gaze_angle_y
of_gaze_distance
of_gaze_distance_x
of_gaze_distance_y
of_pose_Rx
of_pose_Ry
of_pose_Rz
of_pose_Tx
of_pose_Ty
of_pose_Tz
of_pose_distance
of_success
of_ts_success
op_num_people


In [33]:
# Audio Columns

a_cols = []
for i in data[0].columns:
    if 'a_' in i and '_change' not in i and '_var' not in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
for i in a_cols:
    print(i)

a_harmonicity
a_intensity
a_mfcc_0
a_mfcc_1
a_pitch_frequency
a_pitch_strength


In [34]:
# ROS Columns

ros_cols = []
for i in data[0].columns:
    if 'ros_' in i and '_change' not in i and '_var' not in i:
        ros_cols.append(i)
        
ros_cols = sorted(ros_cols)
for i in ros_cols:
    print(i)

ros_aptitude
ros_diff_1
ros_diff_2
ros_diff_3
ros_diff_4
ros_diff_5
ros_games_session
ros_in_game
ros_mistakes_game
ros_mistakes_session
ros_skill_EM
ros_skill_NC
ros_skill_OS
ros_ts_attempt
ros_ts_game_start
ros_ts_robot_talked


In [35]:
# For Window Only:
non_window_features = []
window_features = []
for i in data[0].columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
for i in window_features:
    print(i)

a_harmonicity_var
a_intensity_var
a_mfcc_0_var
a_mfcc_1_var
a_pitch_frequency_var
a_pitch_strength_var
of_AU01_c_change
of_AU02_c_change
of_AU04_c_change
of_AU05_c_change
of_AU06_c_change
of_AU07_c_change
of_AU09_c_change
of_AU10_c_change
of_AU12_c_change
of_AU14_c_change
of_AU15_c_change
of_AU17_c_change
of_AU20_c_change
of_AU23_c_change
of_AU25_c_change
of_AU26_c_change
of_AU28_c_change
of_AU45_c_change
of_confidence_var
of_gaze_0_x_var
of_gaze_0_y_var
of_gaze_0_z_var
of_gaze_1_x_var
of_gaze_1_y_var
of_gaze_1_z_var
of_gaze_angle_x_var
of_gaze_angle_y_var
of_gaze_distance_var
of_gaze_distance_x_var
of_gaze_distance_y_var
of_pose_Rx_var
of_pose_Ry_var
of_pose_Rz_var
of_pose_Tx_var
of_pose_Ty_var
of_pose_Tz_var
of_pose_distance_var
of_success_change
of_ts_success_var
op_num_people_change
ros_aptitude_var


In [36]:
# Columns where NaNs filled with max value
nan_max_cols = ['of_gaze_0_x',
'of_gaze_0_y',
'of_gaze_0_z',
'of_gaze_1_x',
'of_gaze_1_y',
'of_gaze_1_z',
'of_gaze_angle_x',
'of_gaze_angle_y',
'of_gaze_distance',
'of_gaze_distance_x',
'of_gaze_distance_y',
'of_pose_Rxv',
'of_pose_Ry',
'of_pose_Rz',
'of_pose_Tx',
'of_pose_Ty',
'of_pose_Tz',
'of_pose_distance']

In [37]:
# Key Features (from feature analysis) 
# Note: Timestamp automatically included

key_features = [
'op_num_people',
'of_pose_distance',
'of_gaze_distance',
'of_confidence',
'ros_mistakes_session',
'ros_ts_robot_talked']

### Filter Feature Set (Optional)

In [38]:
# always include basic_cols, add desired group of features 
# features_to_keep = all_data.columns

if use_key_features:
    features_to_keep = basic_cols + key_features
else:
    features_to_keep = basic_cols + of_cols + ros_cols + a_cols + window_features

for i,d in enumerate(data):
    data[i] = d[features_to_keep]

# 3 -- Scenario Based Modeling

- Open Face Success / Failure
- Robot Talking / Not Talking
- First 10 Minutes / After 10 Minutes

In [39]:
# Create Separate Models for Different Scenarios

# for i,d in enumerate(data):
#     # Open Face Success/Failure
#     data[i] = d.loc[d['of_success']==1]
#     data[i] = d.loc[d['of_success']==0]
    
#     # Robot Talking/Not Talking
#     data[i] = d.loc[d['ros_ts_robot_talked']==0]
#     data[i] = d.loc[d['ros_ts_robot_talked']>0]
    
#     # First 10 Min/After 10 Min
#     data[i] = d.loc[d['timestamp']<=(10*60)]
#     data[i] = d.loc[d['timestamp']>(10*60)]

# 4 -- Modeling

In [40]:
# Helper Function: Split a Participant's Data Chronologically 

def part_split(split, split_size):
    split = split.sort_values(['session_num', 'timestamp'], ascending=[True, True])
    bogus = split['engagement']

    split_train, split_test, bogus1, bogus2 = train_test_split(split, bogus, test_size=split_size, shuffle=False)
    return split_test, split_train

In [41]:
# Function: Formulate Train-Test Split 
# Includes Preprocessing & Shuffle 

# train_part: an array of participants 
# test_part: an array of participants
# split_size: how much of each test participant's data to use for validation
def split(train_part, test_part, split_size):
    train_data = []
    for p in train_part:
        i = part.index(p)
        train_data.append(data[i].copy())
    
    val_data = []
    test_data = []
    for p in test_part:
        i = part.index(p)
        if split_size > 0:
            p_test, p_train = part_split(data[i].copy(), split_size)
            val_data.append(p_train)
            test_data.append(p_test)
        else:
            test_data.append(data[i].copy())
        
    train_data = pd.concat(train_data, ignore_index=True, sort=True)
    val_data = pd.concat(val_data, ignore_index=True, sort=True)
    test_data = pd.concat(test_data, ignore_index=True, sort=True)
    
    # Sort Training & Validation Data 
    train_data = train_data.sample(frac=1)
    train_data = train_data.sample(frac=1)
    train_data = train_data.sample(frac=1)
    val_data = val_data.sample(frac=1)
    val_data = val_data.sample(frac=1)
    val_data = val_data.sample(frac=1)

    X_train = train_data.drop(columns=['engagement', 'session_num', 'participant'])        
    X_val = val_data.drop(columns=['engagement', 'session_num', 'participant'])  
    X_test = test_data.drop(columns=['engagement', 'session_num', 'participant'])        

    y_train = train_data['engagement']
    y_val = val_data['engagement']
    y_test = test_data['engagement']

    # Preprocessing: Standardization
    # x' = ( x - mean(x) ) / ( stdev(x) )
    # => x' is z-score, NaN's filled with min_val
    for c in X_train.columns:
        mean = np.nanmean(X_train[c])
        std = np.nanstd(X_train[c])

        if std == 0:
            X_train[c] = (X_train[c]-mean)
            X_val[c] = (X_val[c]-mean)
            X_test[c] = (X_test[c]-mean)
        else:
            X_train[c] = (X_train[c]-mean)/(std)
            X_val[c] = (X_val[c]-mean)/(std)
            X_test[c] = (X_test[c]-mean)/(std)

        if c not in nan_max_cols:
            min_val = np.nanmin(X_train[c])

            X_train[c] = X_train[c].fillna(min_val)
            X_val[c] = X_val[c].fillna(min_val)
            X_test[c] = X_test[c].fillna(min_val)
        else:
            max_val = np.nanmax(X_train[c])

            X_train[c] = X_train[c].fillna(max_val)
            X_val[c] = X_val[c].fillna(max_val)
            X_test[c] = X_test[c].fillna(max_val) 

    return X_train, y_train, X_val, y_val, X_test, y_test

In [42]:
# MODEL HERE
# Multiple Participants Approach 2: 
# Approach 2: Use Small % of Test Participant's Data (Chronological Split)

# Inputs: X_train, y_train, X_val, y_val, X_test, y_test

# Output: Accuracy Metrics as Dictionary 

# IMPORTS HERE
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.callbacks import EarlyStopping

def model(X_train, y_train, X_val, y_val, X_test, y_test):
    # Model Here: must create pred & scores arrays
    
    model = Sequential()
    model.add(Dense(1024, input_dim=len(X_train.columns), activation='tanh'))
    model.add(Dense(256, activation='tanh'))
    model.add(Dense(128, activation='tanh'))
    model.add(Dense(64, activation='tanh'))
    model.add(Dense(16, activation='tanh'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=0)
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1000, verbose=0, callbacks=[es])

    
    scores = model.predict(X_test)
    scores = [x[0] for x in scores]
    pred = [int(round(x)) for x in scores]
    
    # Evaluation
    results = {}
    accuracy = accuracy_score(y_test, pred)
    results['accuracy'] = accuracy
    
    try:
        auc = roc_auc_score(y_test, scores)
    except:
        auc = np.nan
    results['auc'] = auc

    precision = metrics.precision_score(y_test, pred, average=None)
    recall = metrics.recall_score(y_test, pred, average=None)
    f1 = metrics.f1_score(y_test, pred, average=None)

    results['precision_0'], results['precision_1'] = precision[0], precision[1]
    results['recall_0'], results['recall_1'] = recall[0], recall[1]
    results['f1_0'], results['f1_1'] = f1[0], f1[1]
    
    return results

In [43]:
all_results = []
for n in range(1, len(part)):    
    if leave_one_out_run:
        if n != len(part)-1:
            continue
    
    comb = combinations(part, n)
    print()
    
    for c in comb:
        print()
        new_results = {}
        
        # Determine Train - Test Participants
        train_part = [x for x in c]
        test_part = [p for p in part if p not in c]
        print('Train:', train_part, '\tTest:', test_part)

        # Train - Test Split (and preprocessing)
        X_train, y_train, X_val, y_val, X_test, y_test = split(train_part, test_part, test_percentage_to_use)
        #X_train = X_train.reset_index(drop=True)
        #y_train = y_train.reset_index(drop=True)
        #X_val = X_val.reset_index(drop=True)
        #y_val = y_val.reset_index(drop=True)

        # Evaluate on Test  
        results = model(X_train, y_train, X_val, y_val, X_test, y_test)
        print(results)
        results['train_participants'] = ", ".join(str(x) for x in train_part)
        results['test_participants'] = ", ".join(str(x) for x in test_part)
        all_results.append(results)



Train: [5] 	Test: [7, 9, 11, 12, 17, 18]
{'precision_1': 0.6309328968903437, 'f1_0': 0.3754371092508212, 'recall_1': 0.9504828436408466, 'precision_0': 0.7861104947858886, 'f1_1': 0.7584228215427493, 'recall_0': 0.24660680726665274, 'accuracy': 0.6516033692921531, 'auc': 0.8214941595532842}

Train: [7] 	Test: [5, 9, 11, 12, 17, 18]
{'precision_1': 0.7581842576028622, 'f1_0': 0.6646109196444768, 'recall_1': 0.8949952486537852, 'precision_0': 0.7865192658581088, 'f1_1': 0.8209287685826352, 'recall_0': 0.5754220651747154, 'accuracy': 0.7665182940303691, 'auc': 0.8184950488621953}

Train: [9] 	Test: [5, 7, 11, 12, 17, 18]
{'precision_1': 0.7871175368653633, 'f1_0': 0.622621015348288, 'recall_1': 0.8215682342324275, 'precision_0': 0.6493941483597675, 'f1_1': 0.8039739973016068, 'recall_0': 0.5979680696661829, 'accuracy': 0.7419761059089441, 'auc': 0.816956549192253}

Train: [11] 	Test: [5, 7, 9, 12, 17, 18]
{'precision_1': 0.7957344890510949, 'f1_0': 0.734859287054409, 'recall_1': 0.80020

{'precision_1': 0.725730278117894, 'f1_0': 0.49681194864180284, 'recall_1': 0.9217268957345972, 'precision_0': 0.7290438349141246, 'f1_1': 0.8120698091665308, 'recall_0': 0.3767885532591415, 'accuracy': 0.7263442903287098, 'auc': 0.8243561015974855}

Train: [5, 7, 11] 	Test: [9, 12, 17, 18]
{'precision_1': 0.7657844265439202, 'f1_0': 0.7709635636650335, 'recall_1': 0.9077846489632594, 'precision_0': 0.8709759511388218, 'f1_1': 0.830760267987183, 'recall_0': 0.6915538492624773, 'accuracy': 0.8053508184167704, 'auc': 0.8966724320333684}

Train: [5, 7, 12] 	Test: [9, 11, 17, 18]
{'precision_1': 0.753725294594426, 'f1_0': 0.6700838207803173, 'recall_1': 0.8882439382806759, 'precision_0': 0.7850784230606189, 'f1_1': 0.8154743836217072, 'recall_0': 0.5844729644435094, 'accuracy': 0.7633241045163524, 'auc': 0.8562844063218202}

Train: [5, 7, 17] 	Test: [9, 11, 12, 18]
{'precision_1': 0.7502137604742632, 'f1_0': 0.6809003843103765, 'recall_1': 0.9394004282655246, 'precision_0': 0.8679626749611

{'precision_1': 0.7590581920214713, 'f1_0': 0.5727462582666203, 'recall_1': 0.928379588182632, 'precision_0': 0.7741707833450953, 'f1_1': 0.8352238405262099, 'recall_0': 0.4544952354647148, 'accuracy': 0.7621700169532575, 'auc': 0.8368274938159888}

Train: [9, 17, 18] 	Test: [5, 7, 11, 12]
{'precision_1': 0.8300807043286867, 'f1_0': 0.6913294797687861, 'recall_1': 0.8196175021732831, 'precision_0': 0.6836890243902439, 'f1_1': 0.8248159218488007, 'recall_0': 0.6991426344505066, 'accuracy': 0.7764859082876011, 'auc': 0.8401822617939051}

Train: [11, 12, 17] 	Test: [5, 7, 9, 18]
{'precision_1': 0.8512614019229189, 'f1_0': 0.7653836218031517, 'recall_1': 0.7913674560733385, 'precision_0': 0.7306174787926613, 'f1_1': 0.8202224949522943, 'recall_0': 0.8036237387436259, 'accuracy': 0.7964316133948984, 'auc': 0.8839111688941607}

Train: [11, 12, 18] 	Test: [5, 7, 9, 17]
{'precision_1': 0.7509649643705463, 'f1_0': 0.7510538359629049, 'recall_1': 0.9285058737151248, 'precision_0': 0.888952245188

{'precision_1': 0.7939572921399364, 'f1_0': 0.4226174674807539, 'recall_1': 0.8655274888558693, 'precision_0': 0.4942564420987271, 'f1_1': 0.8281990521327014, 'recall_0': 0.36911662415951774, 'accuracy': 0.7351920618493943, 'auc': 0.7738208749624529}

Train: [7, 9, 12, 18] 	Test: [5, 11, 17]
{'precision_1': 0.7517800960423912, 'f1_0': 0.46812638580931265, 'recall_1': 0.9153225806451613, 'precision_0': 0.66785290628707, 'f1_1': 0.8255295935994181, 'recall_0': 0.36035843823341157, 'accuracy': 0.7372492640514822, 'auc': 0.800880653076113}

Train: [7, 9, 17, 18] 	Test: [5, 11, 12]
{'precision_1': 0.8007141642577793, 'f1_0': 0.6330544879041374, 'recall_1': 0.9125968992248062, 'precision_0': 0.7563479200432199, 'f1_1': 0.8530024454306676, 'recall_0': 0.5443234836702955, 'accuracy': 0.7900931195033627, 'auc': 0.8430586699940926}

Train: [7, 11, 12, 17] 	Test: [5, 9, 18]
{'precision_1': 0.8, 'f1_0': 0.744178337614983, 'recall_1': 0.9048511347074745, 'precision_0': 0.8308660251665433, 'f1_1': 0

{'precision_1': 0.8127390653583901, 'f1_0': 0.7280595464774103, 'recall_1': 0.9165416354088523, 'precision_0': 0.8253532182103611, 'f1_1': 0.8615249008373733, 'recall_0': 0.6512852276246516, 'accuracy': 0.8164934003037029, 'auc': 0.8775890356918744}

Train: [7, 9, 12, 17, 18] 	Test: [5, 11]
{'precision_1': 0.8174313008609506, 'f1_0': 0.5014588601439408, 'recall_1': 0.8722627737226277, 'precision_0': 0.5594618055555556, 'f1_1': 0.8439573820395739, 'recall_0': 0.4543531899894255, 'accuracy': 0.7623110451636835, 'auc': 0.7807883199258017}

Train: [7, 11, 12, 17, 18] 	Test: [5, 9]
{'precision_1': 0.8039888834395946, 'f1_0': 0.8226921093347086, 'recall_1': 0.9043766090474439, 'precision_0': 0.8846495119787046, 'f1_1': 0.8512332323669407, 'recall_0': 0.7688451898978215, 'accuracy': 0.8382117647058823, 'auc': 0.922684802792621}

Train: [9, 11, 12, 17, 18] 	Test: [5, 7]
{'precision_1': 0.8822469701932525, 'f1_0': 0.7565861690450055, 'recall_1': 0.836230984166408, 'precision_0': 0.7232423924449

In [44]:
results = pd.DataFrame(columns=['train_participants', 'test_participants', 'accuracy', 'auc', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1'])
results = results.append(all_results, ignore_index=True, sort=True)

In [45]:
results.head()

,accuracy,auc,f1_0,f1_1,precision_0,precision_1,recall_0,recall_1,test_participants,train_participants
0,0.651603,0.821494,0.375437,0.758423,0.786110,0.630933,0.246607,0.950483,"7, 9, 11, 12, 17, 18",5
1,0.766518,0.818495,0.664611,0.820929,0.786519,0.758184,0.575422,0.894995,"5, 9, 11, 12, 17, 18",7
2,0.741976,0.816957,0.622621,0.803974,0.649394,0.787118,0.597968,0.821568,"5, 7, 11, 12, 17, 18",9
3,0.770674,0.839599,0.734859,0.797964,0.737572,0.795734,0.732167,0.800206,"5, 7, 9, 12, 17, 18",11
4,0.757401,0.836318,0.617287,0.822416,0.812139,0.740030,0.497843,0.925444,"5, 7, 9, 11, 17, 18",12


In [46]:
results.to_csv(output_file, index=False)